In [ ]:
import sys
import pandas as pd
from pathlib import Path
sys.path.insert(0, '/src')
from eliot import start_action, start_task, to_file, log_message
from shared.utils import get_client_class, get_memory_usage
to_file(sys.stdout)

In [ ]:
processed_path = Path('/data/processed')


In [ ]:
%%time

demo_df = pd.read_parquet(processed_path/'demo_df.parquet')
vitals_df = pd.read_parquet(processed_path/'vitals_df.parquet')
# orders_df = pd.read_parquet(processed_path/'orders_df.parquet')
# alerts_df = pd.read_parquet(processed_path/'alerts_df.parquet')
# meds_df = pd.read_parquet(processed_path/'meds_df.parquet')
rehosp_df = pd.read_parquet(processed_path/'rehosp_df.parquet')
admissions_df = pd.read_parquet(processed_path/'admissions_df.parquet')
diagnosis_df = pd.read_parquet(processed_path/'diagnosis_df.parquet')

In [ ]:
%%time

final_df = demo_df.merge(
    vitals_df,
    how='left',
    left_on=['masterpatientid', 'facilityid', 'censusdate'],
    right_on=['masterpatientid', 'facilityid', 'censusdate']
)
final_df = final_df.merge(
    rehosp_df,
    how='left',
    left_on=['masterpatientid', 'facilityid', 'censusdate'],
    right_on=['masterpatientid', 'facilityid', 'censusdate']
)
final_df = final_df.merge(
    admissions_df,
    how='left',
    left_on=['masterpatientid', 'facilityid', 'censusdate'],
    right_on=['masterpatientid', 'facilityid', 'censusdate']
)
final_df = final_df.merge(
    diagnosis_df,
    how='left',
    left_on=['masterpatientid', 'facilityid', 'censusdate'],
    right_on=['masterpatientid', 'facilityid', 'censusdate']
)
# final_df = final_df.merge(
#     orders_df,
#     how='left',
#     left_on=['masterpatientid', 'facilityid', 'censusdate'],
#     right_on=['masterpatientid', 'facilityid', 'censusdate']
# )
# final_df = final_df.merge(
#     meds_df,
#     how='left',
#     left_on=['masterpatientid', 'facilityid', 'censusdate'],
#     right_on=['masterpatientid', 'facilityid', 'censusdate']
# )
# final_df = final_df.merge(
#     alerts_df,
#     how='left',
#     left_on=['masterpatientid', 'facilityid', 'censusdate'],
#     right_on=['masterpatientid', 'facilityid', 'censusdate']
# )


In [ ]:
if Path.exists(processed_path/'labs_df.parquet'):
    labs_df = pd.read_parquet(processed_path/'labs_df.parquet')
    final_df = final_df.merge(
        labs_df,
        how='left',
        left_on=['masterpatientid', 'facilityid', 'censusdate'],
        right_on=['masterpatientid', 'facilityid', 'censusdate']
    )

In [ ]:
if Path.exists(processed_path/'notes_df.parquet'):
    notes_df = pd.read_parquet(processed_path/'notes_df.parquet')
    final_df = final_df.merge(
        notes_df,
        how='left',
        left_on=['masterpatientid', 'facilityid', 'censusdate'],
        right_on=['masterpatientid', 'facilityid', 'censusdate']
    )

In [ ]:
# drop unwanted columns
final_df.drop(
    final_df.columns[final_df.columns.str.contains('_masterpatientid|_facilityid|_x$|_y$')].tolist()
, axis=1, inplace = True)

if 'patientid' in final_df.columns:
    # If patientid included in the above regex pattern it drops masterpatientid column even
    final_df.drop(
        ['patientid'],
        axis=1,
        inplace=True
    )

In [ ]:
# Write to new parquet file
final_df.to_parquet(processed_path/'final_df.parquet')

In [ ]:
print(get_memory_usage(final_df))

## =============================================================

In [ ]:
final_df.shape

In [ ]:
nan_cols = [i for i in final_df.columns if final_df[i].isna().any() and 'e_' not in i]
nan_cols

In [ ]:
len(nan_cols)
